## Some visualisation on presets values

In [1]:
import os

import numpy as np
from scipy.io import wavfile
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib widget
from IPython.display import Audio

from synth import dexed


In [2]:
dexed_renderer = dexed.Dexed()
dexed_preset_db = dexed.PresetDatabase()
# Visu du dataframe de départ
presets_df = dexed_preset_db.all_presets_df

def get_preset_UID(preset_index):
    return presets_df.iloc[preset_index]['index_preset']

presets_df

,index_preset,index_cart,index_in_cartridge,name,other_names
0,0,0,0,BRITE RDS,None
1,1,0,1,BRITE.FULL,None
2,2,0,2,BRITE-FULL,None
3,3,0,3,BRKN WNGS1,None
4,4,0,4,BROKEROADS,None
...,...,...,...,...,...
30288,393869,12529,18,Hair-kr MS,None
30289,393870,12529,19,DreamySyn,None
30290,393874,12529,23,timbali,None
30291,393876,12529,25,PIES GLOCK,None


In [3]:
params_to_lock = [0, 1, 2, 3, 13]

# Histogrammes de *tous* les paramètres, pour voir ceux qu'on peut éventuellement virer
param_names = dexed_preset_db.get_param_names()
presets_mat = dexed_preset_db.presets_mat
print("{} free params, {} params should be locked".format(presets_mat.shape[1] - len(params_to_lock), len(params_to_lock)))
n_cols = 6
n_rows = 1 + presets_mat.shape[1] // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, n_rows * 1.0), sharex=True)
for i in range(presets_mat.shape[1]):
    param_values = presets_mat[:, i]
    col = i % n_cols
    row = i // n_cols
    # Custom bins for discrete params
    param_cardinality = dexed_renderer.get_param_cardinality(i)
    if param_cardinality > 0:
        bins = [float(i) / param_cardinality for i in range(param_cardinality+1)]
    else:
        bins = 50
    if i in params_to_lock:
        color = 'k'
    else:
        color = 'C0'
    axes[row, col].hist(param_values, bins=bins, fc=color)
    axes[row, col].set_xlim([-0.1, 1.1])
    axes[row, col].set_title("{} {}".format(i, param_names[i]), fontsize=8)
fig.tight_layout()

150 free params, 5 params should be locked


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Paramètres fixes
* Les OP Switch sont toujours à ON

### Paramètres à verrouiller
* 13 transpose (middle C) devrait rester à sa valeur centrale. Pas besoin d'aller perturber tout le pitch
* 0, 1, 2, 3 (params généraux de filtre et de volume) peuvent aussi toujours rester à leurs valeur de défaut

### Paramètres qui risquent de poser problème
* Tous les params liés à la key (si 1 seule note pour tout le NN...)

In [4]:
# Infos sur les paramètres potentiellement inutilisés
for i in range(6):  # synthés qui ont 1 des 6 oscillateurs éteint ?
    opx_switch_values = presets_mat[:, 44 + 22 * i]
    print("OP{} est OFF pour {} preset(s)".format(i+1, opx_switch_values[np.where(opx_switch_values < 0.5)].shape[0]))

OP1 est OFF pour 0 preset(s)
OP2 est OFF pour 0 preset(s)
OP3 est OFF pour 0 preset(s)
OP4 est OFF pour 0 preset(s)
OP5 est OFF pour 0 preset(s)
OP6 est OFF pour 0 preset(s)


In [5]:
# Répartition des algorithmes
preset_algos = presets_mat[:, 4]
preset_algos = np.asarray(np.round(1.0 + preset_algos * 31.0), dtype=np.int)
fig, ax = plt.subplots(1, 1, figsize=(9, 2))
ax.hist(preset_algos, bins=[0.5 + i for i in range(33)], ec='k')
_ = ax.set_title("Histogramme des algorithmes utilisés pour l'ensemble des presets")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Dexed-generated wav file


In [28]:
import librosa
import importlib
from data import dataset
import soundfile
importlib.reload(dataset)
importlib.reload(dexed)

limited_preset_indexes = dexed_preset_db.get_preset_indexes_for_algorithm(5)

def render_and_plot_audio(preset_idx, load_pre_rendered_wav=False):
    midi_note = 60
    midi_vel = 100
    #idx = preset_indexes[1363]
    idx = preset_idx
    if not load_pre_rendered_wav:
        # Nouvelle instance de dx7 à chaque nouveau rendu (pour éviter hanging notes...)
        dexed_renderer = dexed.Dexed()

        preset = dexed_preset_db.get_preset_values(idx, plugin_format=True)
        dexed_renderer.assign_preset(preset)
        dexed_renderer.set_default_general_filter_and_tune_params()
        dexed_renderer.set_all_oscillators_on()
        dexed_renderer.prevent_SH_LFO()
        audio = dexed_renderer.render_note(midi_note, midi_vel, normalize=False)  # 60: . 32: E3
        #print(dexed_renderer.current_preset[12])

        Fs = dexed_renderer.Fs
    else:
        audio, Fs = dataset.DexedDataset.get_wav_file(get_preset_UID(idx), midi_note, midi_vel)
    t = np.linspace(0.0, len(audio)/Fs, num=len(audio), endpoint=True)
    fig, axes = plt.subplots(2, 1, figsize=(9, 4))
    axes[0].plot(t, audio)
    axes[0].set_title("Preset {} (uid={}) '{}' (algo {})".format(idx, get_preset_UID(idx),
                                                        dexed_preset_db.get_preset_name(idx),
                                                        dexed_preset_db._preset_algos[idx]))
    rms = librosa.feature.rms(y=audio)
    times = librosa.times_like(rms)
    axes[1].semilogy(times, rms[0], label='RMS Energy (max {:.3f})'.format(rms[0].max()))
    axes[1].legend()
    
    return audio, Fs, rms[0].max()

audio, Fs, _ = render_and_plot_audio(1144)
Audio(audio, rate=Fs)

<ipython-input-28-006d93e4a9f6>:31: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(2, 1, figsize=(9, 4))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:


# Recherche de tous les presets à volume de sortie trop faible - lecture depuis .wav pré-rendus
rms_energies = list()
low_amplitude_indexes = list()

for idx in range(presets_mat.shape[0]):
    audio, Fs = soundfile.read(data.dataset.DexedDataset.get_wav_file_path(get_preset_UID(idx), 60, 100))
    
    rms_energies.append(librosa.feature.rms(y=audio)[0].max())
    if rms_energies[-1] < 0.03:  # -30dB
        low_amplitude_indexes.append(idx)
        
    if idx%5000 == 0:
        print("Preset {}/{}".format(idx, presets_mat.shape[0]))

print("{} low-energy presets".format(len(low_amplitude_indexes)))

Preset 0/30293
Preset 5000/30293
Preset 10000/30293
Preset 15000/30293
Preset 20000/30293
Preset 25000/30293
Preset 30000/30293
769 low-energy presets


In [10]:
# TODO histrogramme des RMS ENERGY
fig, ax = plt.subplots(1, 1)
ax.hist(rms_energies, bins=200)
ax.set(title="Répartition des énergies RMS max. pour l'ensemble des presets")

print(low_amplitude_indexes)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[79, 95, 174, 184, 254, 366, 368, 369, 372, 384, 415, 422, 539, 586, 726, 745, 746, 806, 807, 925, 976, 1043, 1044, 1058, 1059, 1322, 1328, 1329, 1338, 1369, 1423, 1444, 1445, 1475, 1480, 1506, 1547, 1658, 1660, 1706, 1797, 1879, 1880, 1886, 1986, 1987, 2046, 2139, 2148, 2157, 2179, 2271, 2272, 2327, 2340, 2358, 2479, 2483, 2490, 2528, 2534, 2562, 2584, 2591, 2624, 2633, 2655, 2841, 2878, 2950, 2977, 2978, 3002, 3025, 3031, 3039, 3060, 3171, 3222, 3230, 3299, 3394, 3395, 3396, 3466, 3532, 3578, 3598, 3607, 3680, 3768, 3782, 3796, 3821, 3834, 3867, 3930, 3966, 3998, 4006, 4021, 4071, 4122, 4266, 4291, 4296, 4297, 4326, 4329, 4375, 4410, 4414, 4427, 4490, 4592, 4630, 4708, 4725, 4735, 4748, 4766, 4771, 4834, 4862, 4871, 4948, 5030, 5035, 5043, 5123, 5145, 5208, 5245, 5246, 5258, 5289, 5308, 5365, 5366, 5475, 5540, 5592, 5694, 5784, 5873, 5876, 5879, 5937, 5938, 5970, 6056, 6104, 6139, 6140, 6182, 6255, 6317, 6328, 6350, 6357, 6410, 6421, 6437, 6467, 6468, 6469, 6473, 6553, 6566, 6585, 67

In [9]:
#audio, Fs, _ = render_and_plot_audio(low_amplitude_indexes[97])
#Audio(audio, rate=Fs, normalize=True)

In [10]:
# !!! DANGER ZONE !!!
import sqlite3

# Deletion of low-RMS presets
low_amplitude_preset_UIDs = dexed_preset_db.all_presets_df.iloc[low_amplitude_indexes]['index_preset'].values
#print(low_amplitude_preset_UIDs)
conn = sqlite3.connect('./synth/dexed_presets.sqlite', detect_types=sqlite3.PARSE_DECLTYPES)
cur = conn.cursor()
for uid in low_amplitude_preset_UIDs:
    # Cursor treated as an iterator over results. execute(...) requires the int to be re-casted....
    #for row in cur.execute("SELECT * FROM preset WHERE index_preset=?", (int(uid),)):
    #    print(row)
    #cur.execute("DELETE FROM preset WHERE index_preset=?", (int(uid),))  # Un-comment to re-activate deletion
    pass
conn.commit()
conn.close()

# Preset aléatoire !

Presets peut-être à virer (ne ressemble vraiment à rien, params aléatoires...) : 73561, 

In [ ]:

uid = -1
load_pre_rendered_wav = True

if uid == -1:  # random preset
    #audio, Fs, _ = render_and_plot_audio(np.random.randint(0, dexed_preset_db.presets_mat.shape[0]), load_pre_rendered_wav=load_pre_rendered_wav)
    audio, Fs, _ = render_and_plot_audio(limited_preset_indexes[np.random.randint(0, len(limited_preset_indexes))], load_pre_rendered_wav=load_pre_rendered_wav)
    #audio, Fs, _ = render_and_plot_audio(low_amplitude_indexes[np.random.randint(0, len(low_amplitude_indexes))], load_pre_rendered_wav=load_pre_rendered_wav)
else:  # if uid is >=0, we search for that particular preset! To better listen to it
    audio, Fs, _ = render_and_plot_audio(presets_df.loc[presets_df['index_preset'] == uid].index[0], load_pre_rendered_wav=load_pre_rendered_wav)

Audio(audio, rate=Fs, normalize=True)

In [ ]:
# .wav pré-rendus
